In [65]:
using LinearAlgebra

In [66]:
Mx = [0  -5  0   0    0  -3//16  -3//8   0;
     1  0   0   0    0   0       0       0;
     0  0   0  -5    0   0       0       0;
     0  0   1   3//20 0   0       0       3//40;
     0  0   0   0    0   5//2    0       0;
     0  0   0  -2    0   0       0      -1;
     0  2   0   0    1   0       0       0;
     0  0   0  -3//10 0  -3//16  -3//8  -3//20]

8×8 Matrix{Rational{Int64}}:
 0//1  -5//1  0//1   0//1   0//1  -3//16  -3//8   0//1
 1//1   0//1  0//1   0//1   0//1   0//1    0//1   0//1
 0//1   0//1  0//1  -5//1   0//1   0//1    0//1   0//1
 0//1   0//1  1//1   3//20  0//1   0//1    0//1   3//40
 0//1   0//1  0//1   0//1   0//1   5//2    0//1   0//1
 0//1   0//1  0//1  -2//1   0//1   0//1    0//1  -1//1
 0//1   2//1  0//1   0//1   1//1   0//1    0//1   0//1
 0//1   0//1  0//1  -3//10  0//1  -3//16  -3//8  -3//20

In [67]:
My = [0  0  0     -1  0  0       0       -1//2;
     0  0  0     0   0  1//2    0       0;
     1  0  0     0   0  0       0       0;
     0  1  0     0   0  3//80   3//40   0;
     0  0  0     0   0  -1      0       20//3;
     0  0  0     0   0  -1//2   -1      0;
     0  0  8//3  0   0  -20//3  0       0;
     0  0  0     -1  1  -3//40  -3//20  -1//2]

8×8 Matrix{Rational{Int64}}:
 0//1  0//1  0//1  -1//1  0//1    0//1    0//1   -1//2
 0//1  0//1  0//1   0//1  0//1    1//2    0//1    0//1
 1//1  0//1  0//1   0//1  0//1    0//1    0//1    0//1
 0//1  1//1  0//1   0//1  0//1    3//80   3//40   0//1
 0//1  0//1  0//1   0//1  0//1   -1//1    0//1   20//3
 0//1  0//1  0//1   0//1  0//1   -1//2   -1//1    0//1
 0//1  0//1  8//3   0//1  0//1  -20//3    0//1    0//1
 0//1  0//1  0//1  -1//1  1//1   -3//40  -3//20  -1//2

In [68]:
Mz = [0  0  0  0       0  -3//32   -3//16  0;
0  0  0  0       0  -3//40   0       1//2;
0  0  0  0       0  3//16    0       0;
0  0  0  3//40   0  0        0       3//80;
1  0  0  0       0  7//5     5//2    -1;
0  0  0  -1      1  0        0       -1//2;
0  1  0  0       0  1        0       -20//3;
0  0  1  -3//20  0  -83//32  -3//16  -3//40]

8×8 Matrix{Rational{Int64}}:
 0//1  0//1  0//1   0//1   0//1   -3//32  -3//16    0//1
 0//1  0//1  0//1   0//1   0//1   -3//40   0//1     1//2
 0//1  0//1  0//1   0//1   0//1    3//16   0//1     0//1
 0//1  0//1  0//1   3//40  0//1    0//1    0//1     3//80
 1//1  0//1  0//1   0//1   0//1    7//5    5//2    -1//1
 0//1  0//1  0//1  -1//1   1//1    0//1    0//1    -1//2
 0//1  1//1  0//1   0//1   0//1    1//1    0//1   -20//3
 0//1  0//1  1//1  -3//20  0//1  -83//32  -3//16   -3//40

In [69]:
function random_linear_combination(matrices) 
    n = length(matrices)
    coeffs = randn(n)
    coeffs /= sum(coeffs)
    result = zeros( size(matrices[1]))
    for i in 1:n
        result += coeffs[i] * matrices[i]
    end
    return result
end

random_linear_combination (generic function with 1 method)

In [71]:
function XYS(Σ,Z,Δ)
    n, m = size(Z)
    S = diagm(diag(Δ-Z*Σ))
    X = [i != j ? (-Δ[i,j]+ Z[i,j] * Σ[j,j])/(Σ[i,i]-Σ[j,j]) : 0 for i in 1:n , j in 1:n]
    Y = [i != j ? (-Δ[i,j]- Z[i,j] * Σ[i,i])/(Σ[i,i]-Σ[j,j]) : -Z[i,i] for i in 1:n , j in 1:n]
    return X,Y,S
end

XYS (generic function with 1 method)

In [72]:
function iteracionNewton(E,F,Σ,M,I_n)
    Z = F*E-I_n
    Δ = F * M * E - Σ
    X,Y,S = XYS(Σ,Z,Δ)
    Em1 = E * (I_n + X)
    Fm1 = (I_n + Y) * F
    Σm1 = Σ + S
    return Em1,Fm1,Σm1
end

iteracionNewton (generic function with 2 methods)

In [73]:
function newTonSimul(matrices)
    M = random_linear_combination(matrices)
    n, m = size(matrices[1])
    Li, Ei = eigen(matrices[1])
    Fi = inv(Ei)
    Σi = diagm(Li)
    I_n = Matrix{Float64}(I, n, n)
    err = opnorm((Fi*M*Ei-Σi), Inf)
    numbern = 0
    println(numbern)
    println(err)
    while err > 1^(-20) 
        Ei,Fi,Σi = iteracionNewton(Ei,Fi,Σi,M,I_n)
        err = opnorm((Fi*M*Ei-Σi), Inf)
        numbern+=1
        println(Ei)
        println(err)
    end
    return Ei,Fi,numbern
end

newTonSimul (generic function with 1 method)

In [74]:
E,F = newTonSimul([Mx,My,Mz])

0
4.006162828694359


Any[0.0527274302651615 + 6.866959282428205e-18im 0.005766300683815913 + 0.04752411334956895im 0.0057663006838159425 - 0.047524113349568936im -0.3222833576557941 + 0.3212018809754391im -0.3222833576557874 - 0.3212018809754214im 0.3018095693793472 - 0.3252338534540271im 0.301809569379361 + 0.32523385345403516im -0.045880974026681544 - 7.757029242849818e-18im; -0.04789102505995175 - 3.3994319842018035e-19im -0.05119229538436623 + 0.0017120312663656643im -0.05119229538436622 - 0.0017120312663656556im -0.14809916776482243 - 0.13927029984584577im -0.14809916776481438 + 0.13927029984584313im 0.1494146692452905 + 0.12944321585377544im 0.14941466924529428 - 0.12944321585378146im -0.047509974348069424 - 1.7505608886031207e-17im; -0.018320842181265687 - 4.9762787088773114e-18im 0.0028236235491310192 + 0.020173079076706707im 0.002823623549131021 - 0.0201730790767067im 0.6919332987262201 - 6.2645964891249315e-12im 0.6919332987262458 + 6.275247848423005e-12im 0.6543580868891102 - 3.7161072853307835e

(Any[0.0527274302651615 + 6.866959282428205e-18im 0.005766300683815913 + 0.04752411334956895im … 0.301809569379361 + 0.32523385345403516im -0.045880974026681544 - 7.757029242849818e-18im; -0.04789102505995175 - 3.3994319842018035e-19im -0.05119229538436623 + 0.0017120312663656643im … 0.14941466924529428 - 0.12944321585378146im -0.047509974348069424 - 1.7505608886031207e-17im; … ; 0.6555225171630598 + 1.8602417054521935e-17im 0.6861866286348455 + 1.1317889382658646e-15im … -0.24169685203590316 - 0.16045029197606758im 0.6179989175052418 + 2.017951290017056e-16im; 0.31156776494367855 - 1.138096406715705e-18im -0.015585585517088411 - 0.22554936012437926im … 0.06495531335087272 - 0.012908057815983705im -0.2517785503653263 - 5.531288878242687e-17im], ComplexF64[0.1153250994188654 - 1.8629035976330813e-16im -0.12697151772840098 - 1.06275039415984e-16im … 0.35820978915451185 - 1.6972442647797744e-16im 0.9363671068932031 + 3.3758315073093715e-16im; 0.18863275855630943 - 0.03067326534653564im -0

In [78]:
diag(F*Mz*E)

8-element Vector{Any}:
   -2.821182227030937 + 2.465190328815662e-31im
 -0.27400458047763915 + 2.1991272590283963im
 -0.27400458047763904 - 2.199127259028395im
   0.0724226556852253 - 0.0021032191277763233im
   0.0724226556852253 + 0.00210321912777632im
  0.07636377398501654 - 0.008344102199833814im
  0.07636377398501654 + 0.008344102199833812im
   3.0716185286457307 - 2.465190328815662e-30im

In [80]:
eigen(Mz).values

8-element Vector{ComplexF64}:
 -2.8211822270309357 + 0.0im
 -0.2740045804776395 - 2.1991272590283937im
 -0.2740045804776395 + 2.1991272590283937im
  0.0724226556852254 - 0.0021032191277764816im
  0.0724226556852254 + 0.0021032191277764816im
 0.07636377398501663 - 0.008344102199833713im
 0.07636377398501663 + 0.008344102199833713im
   3.071618528645733 + 0.0im